В рамках конкурса вам нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического 
врачебного осмотра. Датасет сформирован и 100.000 реальных клинических анализов, и в нём используются признаки,
которые можно разбить на 3 группы:
    
Объективные признаки:

     - Возраст
     - Рост
     - Вес
     - Пол

Результаты измерения:

     - Артериальное давление верхнее и нижнее
     - Холестерин
     - Глюкоза

Субъективные признаки:

     - Курение
     - Употребление Алкоголя
     - Физическая активность

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма,
    выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.
 
Последняя группа признаков не имеет однозначной интерпретации и более того, собрана со слов самого больного.
Поэтому, для части данных в тестовой выборке мы специально исключили эти параметры. Участникам придется либо
предсказать их самостоятельно для некоторых случаев, либо вовсе игнорировать.

import os
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import numpy

import seaborn as sns
sns.set(style="white", color_codes=True)

%matplotlib inline

data = pd.read_csv('data/train.csv',sep=";")

#пропуски в данных
data.info()

data['gender'].value_counts()

data.describe()

Можно заметить, что имеют место некорректные значения. Работу по коррекции значений можно проводить исходя из общих соображений,
также привлекать автоматизированные методы. Выбор тех или иных подходов может быть обусловен условием задачи. Иногда можно просто
избавиться от неправильных значений без потерь.

# Таблицы


#среднее значение cardio зависит от уровня холестерина
data.groupby(['gender','cholesterol'])['cardio'].aggregate('mean').unstack()

Между мужчинами и женщинам разницы нет, но зато есть разница у тех, кого холестерин в норме, выше нормы и много выше нормы

#среднее значение cardio зависит от уровня сахара
data.groupby(['gender','gluc'])['cardio'].aggregate('mean').unstack()

Казалось бы логично предположить, что среди людей с сахаром много выше нормы, будет больше больных СС3, 
но это верно только для мужчин

#курение
data.groupby(['smoke'])['cardio'].aggregate('mean')

#алкоголь
data.groupby(['alco'])['cardio'].aggregate('mean')

Люди, которые курят и пьют в среднем имеют меньше проблем с сердцем???

data.corr()

# Потенциальные проблемы

## нижние и верхнее давление

In [ ]:
#отобразим верхнее и нижнее давление
plt.figure(figsize=(12,8))
plt.scatter(data.ap_lo,data.ap_hi,s=9,c=data.cardio,cmap='seismic')
plt.colorbar()
plt.plot([50,200],[50,200],'--',linewidth=0.8)
plt.xlabel('ap_lo')
plt.ylabel('al_hi')

## вес и рост

In [ ]:
#
plt.figure(figsize=(12,8))
plt.scatter(data.weight,data.height,s=9,c=data.cardio,cmap='seismic')
plt.colorbar()
plt.plot([50,200],[50,200],'--',linewidth=0.8)
plt.xlabel('weight')
plt.ylabel('height')

Что сразу бросается в глаза? Есть ли на графике нереалистичные данные? Избавиться от наблюдения или принять его верным?

# Спосбо нахождения аномалий

In [ ]:
#диаграмма размаха
f, axes = plt.subplots(2,1)

sns.boxplot(data.weight, palette="PRGn",ax=axes[0])
sns.distplot(data.weight, ax=axes[1])

In [ ]:
f, axes = plt.subplots(2,1)

sns.boxplot(data.height, palette="PRGn",ax=axes[0])
sns.distplot(data.height, ax=axes[1])

In [ ]:
f, axes = plt.subplots(2,1)

sns.boxplot(data.age, palette="PRGn",ax=axes[0])
sns.distplot(data.age, ax=axes[1])

# иллюстация для диаграммы размах

d=[180,147,200,150,130,165]


sorted(d)

In [ ]:
#среднее

In [ ]:
#медиана

d=[180,147,200,150,130,165,300]

# произведем очистку данных

In [57]:
mask = (data.ap_hi==1)&(data.ap_lo>100)
data.loc[mask,'ap_hi']=(data.loc[mask,'ap_hi']*100+(data.loc[mask,'ap_lo']//100))
data.loc[mask,'ap_lo'] = (data.loc[mask,'ap_lo'].mod(100)//10)*10
data.loc[data.ap_hi==14900,'ap_hi']=140
data.loc[data.ap_hi>4000,'ap_hi']//=100
mask = (data.ap_lo==0)&(data.ap_hi>=300)
data.loc[mask,'ap_hi'],data.loc[mask,'ap_lo']=data.loc[mask,'ap_hi']//10,data.loc[mask,'ap_hi'].mod(10)*10
data.loc[data.ap_hi.isin([400,401]),'ap_hi']=140
data.loc[data.ap_hi==701,'ap_hi']=170
data.loc[data.ap_hi==1420,'ap_hi']=140
data.loc[data.ap_hi==1620,'ap_hi']=160
data.loc[data.ap_hi.isin([1130,1110]),'ap_hi']=110
data.loc[data.ap_hi==960,'ap_hi']=90
data.loc[data.ap_hi==300,'ap_hi']=10
data.loc[data.ap_hi>=240,'ap_hi']=(data.loc[data.ap_hi>=240,'ap_hi']//100)*10

data.loc[data.ap_lo==4100,'ap_lo']=140
data.loc[data.ap_lo==10000,'ap_lo']=100
data.loc[data.ap_lo==5700,'ap_lo']=75
data.loc[data.ap_lo==6800,'ap_lo']=80
data.loc[data.ap_lo==4700,'ap_lo']=70
data.loc[data.ap_lo==1100,'ap_lo']=110

data.loc[data.ap_lo>=4000,'ap_lo']=(data.loc[data.ap_lo>=4000,'ap_lo']//1000)*10
data.loc[data.ap_lo==1900,'ap_lo']=90
data.loc[data.ap_lo==1211,'ap_lo']=120
data.loc[data.ap_lo>=1200,'ap_lo']//=10
data.loc[data.ap_lo>=1100,'ap_lo']=(data.loc[data.ap_lo>=1100,'ap_lo']//100)*10
data.loc[data.ap_lo>=1000,'ap_lo']=100
data.loc[data.ap_lo==570,'ap_lo']=75
data.loc[data.ap_lo.isin([850,855]),'ap_lo']=85
data.loc[data.ap_lo>=300,'ap_lo']=(data.loc[data.ap_lo>=300,'ap_lo']//100)*10


#диаграмма размаха
f, axes = plt.subplots(2,1)

sns.boxplot(data.weight, palette="PRGn",ax=axes[0])
sns.distplot(data.weight, ax=axes[1])

f, axes = plt.subplots(2,1)

sns.boxplot(data.height, palette="PRGn",ax=axes[0])
sns.distplot(data.height, ax=axes[1])

In [ ]:
data.corr()

In [ ]:
sns.pairplot(data[['age','height','weight']],size=3)

data.describe()

# посмотрим на распределение признаков в наборе данных

In [ ]:
#рост,вес,возраст
data_long = pd.melt(data[['height','weight','gender']],"gender",var_name="feature")
g= sns.FacetGrid(data_long, col="feature",hue="gender",col_wrap=2,sharex=False,size=6,legend_out=True)
g.map(sns.distplot,"value")

In [ ]:
#сколько мужчин, женищн в соотношении cardio 1/0 у мужчин и женщин
fig, (axis1,axis2)=plt.subplots(1,2,figsize=(15,5))
fig, (axis3,axis4)=plt.subplots(1,2,figsize=(15,5))

sns.countplot(x='gender',data=data,ax=axis1)
sns.countplot(x='cardio',hue="gender",data=data,order=[1,0],ax=axis2)


data_gluc = data[["gluc","cardio"]].groupby(['gluc'],as_index=False).mean()
sns.barplot(x='gluc', y='cardio',data=data_gluc,order=[1,2,3],ax=axis3)

data_gluc = data[["cholesterol","cardio"]].groupby(["cholesterol"],as_index=False).mean()
sns.barplot(x='cholesterol',y='cardio',data=data_gluc,order=[1,2,3],ax=axis4)

In [68]:
#сколько больных в зависимости от алкоголя
fig, (axis1,axis2)=plt.subplots(1,2,figsize=(15,5))

sns.countplot(x='alco',data=data,ax= axis1)
sns.countplot(x='cardio',hue="alco",data=data,order=[1,0],ax=axis2)

#сколько больных в зависимости от курения
fig, (axis3,axis4)=plt.subplot(1,2,figsize=(15,5))

sns.countplot(x='smoke',data=data,axis1)
sns.countplot(x='cardio', hue="smoke",data=data,order=[1,0],ax=axis4)

#сколько занимаются спортом
fig, (axis5,axis6)=plt.subplot(1,2,figsize=(15,5))

sns.countplot(x='active',data=data,axis1)
sns.countplot(x='cardio',hue="active",data=data,order=[1,0],ax=axis2)

SyntaxError: positional argument follows keyword argument (<ipython-input-68-4797eb637b95>, line 4)

In [ ]:
#посмортим на возраст, но сначал преобразуем его
data['age']=data['age']/365.25
sns.distplot(data['age'],label="Age",bins=100)

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(data.ap_lo,data.ap_hi,s=9,c=data.cardio,cmap='seismic')
plt.colorbar()
plt.plot([50,200],[50,200],'--',linewidth=0.8)
plt.xlabel('ap_lo')
plt.ylabel('al_hi')